In [434]:
%matplotlib inline
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
import cvxopt as opt
from cvxopt import blas, solvers


np.random.seed(123)

# Turn off progress printing 
solvers.options['show_progress'] = False

#Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['MSFT', 'T', 'GOOG']
nAssets = len(tickers)
print("Number of assets: %d"%nAssets)

# Define which online source one should use
data_source = 'yahoo'

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2012-01-02'
end_date = '2012-12-31'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
#trainingData = data.DataReader(tickers, data_source, start_date, end_date)['Adj Close']
trainingData = data.get_data_yahoo(tickers, start_date, end_date)

Number of assets: 3


/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


In [435]:
trainingData = trainingData.sort_index(axis=1)
# Getting just the adjusted closing prices. This will return a Pandas DataFrame
# The index in this DataFrame is the major index of the panel_data.
trainingClose = trainingData.ix['Adj Close']
trainingClose = trainingClose.fillna(method='ffill')
#print(trainingClose)

# Opening prices.
trainingStart = trainingData.ix['Open']
trainingStart = trainingStart.fillna(method='ffill')

In [437]:
def calculateReturns(start, close):
    X0 = [start[ticker].iloc[0] for ticker in tickers]
    print("Starting prices: ", X0)

    Returns = []
    # Closing price every day.
    for i, ticker in enumerate(tickers):
        closingPrices = close[ticker]
        # Caclulate returns
        calcReturn = lambda x: float(x - X0[i]) / X0[i]
        Returns.append(closingPrices.apply(calcReturn))
    return Returns
    
def optimalPortfolio(Returns, retMin):
    # Calculate the expected Return for each asset.
    numDays = len(Returns)
    mus = np.asarray(Returns.mean())
    n = len(mus)
    print("Mean returns: ", mus)
    mus = np.reshape(mus, (len(mus), 1))

    Cov = opt.matrix(np.cov(Returns.T))
    print(Cov)
    Mus = opt.matrix(mus)
       
    q = opt.matrix(np.zeros((n, 1)))
    
    # Create constraint matrices
    # w >= 0 and min return constraint.
    G = -opt.matrix(np.concatenate((mus.T, np.eye(n))))   # negative n x n identity matrix
    h = opt.matrix(np.concatenate((-np.ones((1,1))*retMin ,np.zeros((n ,1)))))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    solution = solvers.qp(Cov, q, G, h, A, b)
    portfolio = solution['x']
    
    returns = blas.dot(Mus, portfolio)*numDays
    print("Returns: ", returns)
    risk = np.sqrt(blas.dot(portfolio, Cov*portfolio))
    print("Risk : %f"%risk)
    return (portfolio, Mus)

#returns3 = calculateReturns(trainingStart, trainingClose)
#np.mean(returns3, axis=1))
returns = trainingClose.pct_change()
returns = returns.fillna(method='bfill')
retMin = 0.0001
optimalPortfolio, Mus = optimalPortfolio(returns, retMin)

optimalPortfolio = optimalPortfolio
print( optimalPortfolio)


('Mean returns: ', array([ 0.00036541,  0.00027977,  0.00067091]))
[ 2.05e-04  3.59e-05  2.96e-05]
[ 3.59e-05  1.70e-04  3.99e-05]
[ 2.96e-05  3.99e-05  7.62e-05]

('Returns: ', 0.13720299448738352)
Risk : 0.007855
[ 1.77e-01]
[ 1.74e-01]
[ 6.49e-01]



In [438]:
list(trainingClose.max())



[381.54342700000001, 28.298154999999998, 29.307717999999998]

In [439]:
def calc_kd(close_data):
    n_ticks = len(tickers)
    K = [[50 for _ in tickers]]
    D = [[50 for _ in tickers]]
    low = list(close_data.min())
    high = list(close_data.max())
    ind = 0
    
    
    while ind < close_data.shape[0]:
        row = close_data.iloc[ind]

        if 1:
            # calculate indices
            k_list = [0 for _ in tickers]
            d_list = [0 for i in tickers]
            for t in range(n_ticks):
                if high[t] - low[t] == 0:
                    rsv = 50
                    #print(high[t])
                else:
                    rsv = ((row[t] - low[t])*100.0) / (high[t] - low[t])
                k = rsv*(1.0/3) + K[-1][t]*(2.0/3)

                k_list[t]  = k				
                d = k*(1.0/3) + D[-1][t]*(2.0/3)
#				print('d = ', d)
                d_list[t]= d
		  
#			k_list = np.array(k_list)
#			d_list = np.array(d_list)
			
            K.append(copy.copy(k_list))
            D.append(copy.copy(d_list))

        ind += 1
  
    return K,D
#print(close.shape)
#K,D = calc_kd(close)
#print(K)

def recursiveKD(Kn_, Dn_, Pn, Ln, Hn):
    rsv = float(Pn - Ln)/(Hn - Ln) *100
    Kn = rsv/3 + Kn_*2/3
    Dn = Kn/3 + Dn_*2/3
    return (Kn, Dn)
        

In [440]:
def getTestData():
    start_date = '2013-01-02'
    end_date = '2013-12-31'
    testData = data.get_data_yahoo(tickers, start_date, end_date)
    testData = testData.sort_index(axis=1)


    # Getting just the adjusted closing prices. This will return a Pandas DataFrame
    # The index in this DataFrame is the major index of the panel_data.
    close = testData.ix['Adj Close']
    close = close.fillna(method='ffill')

    # Opening prices.
    start = testData.ix['Open']
    start = start.fillna(method='ffill')
    return (start, close)

testStart, testClose = getTestData()



/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:137: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


In [442]:
testReturns = testClose.pct_change()
testReturns = testReturns.fillna(method="bfill")

def calculateReturn(weights, returns, riskFreeRate):
    returnsSum = np.sum(returns)
    returns = np.matmul(returnsSum, weights)
    
    # Calculate risk free return
    riskFreeWeight = 1 - np.sum(weights)
    riskFreeReturn = riskFreeWeight * riskFreeRate
    return returns + riskFreeReturn

def getNextWindow(returnRates, newRate):
    newReturnRates = returnRates.drop(returnRates.index[0])
    newReturnRates.append(newRate)
    return newReturnRates

def getSignal(K, D):
    "Returns 1 for buy and 0 for sell and -1 for nothing."
    Kn, Dn = K[-1], D[-1]
    Kn_, Dn_ = K[-2], D[-2]
    print(Kn_, Dn_)
    print(Kn, Dn)
    signals = []
    for i in range(nAssets):
        if Kn_[i] <= Dn_[i] and Kn[i] > Dn[i]:
            signals.append(1)
        elif Kn_[i] >= Dn_[i] and Kn[i] < Dn[i]:
            signals.append(0)
        else:
            signals.append(-1)
    assert(len(signals) == nAssets)
    return signals

def conservativeStrategy(returnRates, riskFreeRate):
    portfolio = optimalPortfolio
    dailyReturns = []
    close = trainingClose
    for i in range(10):
        testDay = testReturns.iloc[[i]]
        K, D = calc_kd(close)
        #print(K)
        #print(D)
        signals = getSignal(K, D)
        portfolio = []
        print(signals)
        
        for i in range(nAssets):
            signal = signals[i]
            if signal:
                portfolio.append(optimalPortfolio[i])
            else:
                portfolio.append(0)
        returns = calculateReturn(portfolio, testDay, riskFreeRate)
        dailyReturns.append(returns)
        # Get training Windown by deleting the first row and adding the lates data.
        returnRates = getNextWindow(returnRates, testDay)
        close = getNextWindow(close, testClose.iloc[[i]])
    return dailyReturns

riskFreeRate = 0.0001
dailyReturns = conservativeStrategy(testReturns, riskFreeRate)
print(np.sum(dailyReturns))

([70.733190676920813, 14.251331593416499, 56.598266133174292], [72.292707364914477, 17.264912687197885, 58.44897967827314])
([70.812555639065693, 13.182374769746689, 56.851556079554591], [71.799323456298211, 15.904066714714151, 57.916505145366955])
[-1, -1, -1]
([70.733190676920813, 9.2684931605754031, 56.598266133174292], [72.292707364914477, 12.457192865218207, 58.44897967827314])
([70.812555639065693, 8.1374194639207698, 56.851556079554591], [71.799323456298211, 11.017268398119061, 57.916505145366955])
[-1, -1, -1]
([70.733190676920813, 9.2684931605754031, 56.598266133174292], [72.292707364914477, 12.457192865218207, 58.44897967827314])
([70.812555639065693, 8.1374194639207698, 56.851556079554591], [71.799323456298211, 11.017268398119061, 57.916505145366955])
[-1, -1, -1]
([70.733190676920813, 9.2684931605754031, 56.598266133174292], [72.292707364914477, 12.457192865218207, 58.44897967827314])
([70.812555639065693, 8.1374194639207698, 56.851556079554591], [71.799323456298211, 11.017